neural networks

In [9]:
import pandas as pd
file_path = 'shots_2022.csv'
df_base = pd.read_csv(file_path)
description = df_base.describe()
print(description)

              shotID  arenaAdjustedShotDistance  arenaAdjustedXCord  \
count  122026.000000              122026.000000       122026.000000   
mean    61012.500000                  33.783658           -0.264345   
std     35226.016311                  19.352137           63.973502   
min         0.000000                   1.000000          -99.000000   
25%     30506.250000                  17.000000          -64.000000   
50%     61012.500000                  32.000000           -1.000000   
75%     91518.750000                  48.000000           64.000000   
max    122025.000000                  99.000000           99.000000   

       arenaAdjustedXCordABS  arenaAdjustedYCord  arenaAdjustedYCordAbs  \
count          122026.000000       122026.000000          122026.000000   
mean               61.090874            0.032518              15.483913   
std                18.988244           19.383625              11.660722   
min                 0.000000          -48.000000            

In [4]:
working_df = df_base[['arenaAdjustedShotDistance', 'goal', 'shotType']]
working_df = working_df.iloc[:]
working_df['shotType'] = working_df['shotType'].fillna('UNK')
shot_type_mapping = {'WRIST': 1, 'SLAP': 2, 'TIP': 3, 'SNAP': 4, 'BACK': 5, 'WRAP': 6, 'DEFL': 7, 'UNK': 8}
working_df['shot_type_id'] = working_df['shotType'].map(shot_type_mapping)
working_df = working_df.rename(columns={'arenaAdjustedShotDistance': 'distance', 'goal': 'is_goal'})
working_df = working_df[['distance', 'shot_type_id', 'is_goal',]]

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

X = working_df[['distance', 'shot_type_id']]
y = working_df['is_goal']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class_weights = {0: 0.07, 1: 0.93}  

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=2)) 
model.add(Dense(1, activation='sigmoid')) 

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test), class_weight=class_weights)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Accuracy: {accuracy}')
predictions = model.predict(X_test_scaled)


Epoch 1/10
3051/3051 [==============================] - 6s 2ms/step - loss: 0.0838 - accuracy: 0.5789 - val_loss: 0.6481 - val_accuracy: 0.5718
Epoch 2/10
3051/3051 [==============================] - 5s 2ms/step - loss: 0.0834 - accuracy: 0.5760 - val_loss: 0.6782 - val_accuracy: 0.5501
Epoch 3/10
3051/3051 [==============================] - 5s 2ms/step - loss: 0.0833 - accuracy: 0.5686 - val_loss: 0.6071 - val_accuracy: 0.5980
Epoch 4/10
3051/3051 [==============================] - 5s 2ms/step - loss: 0.0832 - accuracy: 0.5730 - val_loss: 0.6550 - val_accuracy: 0.5638
Epoch 5/10
3051/3051 [==============================] - 6s 2ms/step - loss: 0.0832 - accuracy: 0.5679 - val_loss: 0.6609 - val_accuracy: 0.5600
Epoch 6/10
3051/3051 [==============================] - 6s 2ms/step - loss: 0.0832 - accuracy: 0.5673 - val_loss: 0.6438 - val_accuracy: 0.5695
Epoch 7/10
3051/3051 [==============================] - 5s 2ms/step - loss: 0.0831 - accuracy: 0.5649 - val_loss: 0.6342 - val_accuracy:

In [13]:
def shooter_on_man_advantage(awaySkatersOnIce, homeSkatersOnIce,isHomeTeam):
    if isHomeTeam:
        if homeSkatersOnIce > awaySkatersOnIce:
            return True
        else:
            return False
    else:
        if awaySkatersOnIce > homeSkatersOnIce:
            return True
        else:
            return False

final_df = df_base[['arenaAdjustedShotDistance', 'goal', 'shotType','lastEventCategory','averageRestDifference','shooterTimeOnIce','awaySkatersOnIce', 'homeSkatersOnIce','isHomeTeam']]
final_df = final_df.iloc[:]
final_df['shotType'] = final_df['shotType'].fillna('UNK')
shot_type_mapping = {'WRIST': 1, 'SLAP': 2, 'TIP': 3, 'SNAP': 4, 'BACK': 5, 'WRAP': 6, 'DEFL': 7, 'UNK': 8}
final_df['shot_type_id'] = final_df['shotType'].map(shot_type_mapping)
prior_event_mapping = {'HIT': 0, 'FAC': 1, 'BLOCK': 2, 'SHOT': 3, 'GIVE': 4, 'MISS': 5, 'TAKE': 6, 'DELPEN': 7, 'STOP': 8, 'CHL': 9}
final_df['prior_event_id'] = final_df['lastEventCategory'].map(prior_event_mapping)
final_df['on_man_advantage'] = final_df.apply(lambda row: shooter_on_man_advantage(row['awaySkatersOnIce'], row['homeSkatersOnIce'], row['isHomeTeam']), axis=1)
final_df = final_df.rename(columns={'arenaAdjustedShotDistance': 'shot_distance', 'averageRestDifference': 'rest_difference','shooterTimeOnIce': 'shooter_time_on_ice', 'goal': 'is_goal'})
final_df = final_df[['shot_distance', 'shot_type_id', 'prior_event_id', 'rest_difference','shooter_time_on_ice','on_man_advantage', 'is_goal',]]

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers

X = final_df[['shot_distance', 'shot_type_id','prior_event_id','rest_difference','shooter_time_on_ice','on_man_advantage']]
y = final_df['is_goal']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

class_weights = {0: 0.07, 1: 0.93}  

model = Sequential()
model.add(Dense(64, activation='relu', input_dim=6, kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test), class_weight=class_weights)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Accuracy: {accuracy}')
predictions = model.predict(X_test_scaled)


Epoch 1/10
3051/3051 [==============================] - 10s 3ms/step - loss: 0.1378 - accuracy: 0.5363 - val_loss: 0.6617 - val_accuracy: 0.5376
Epoch 2/10
3051/3051 [==============================] - 8s 2ms/step - loss: 0.0867 - accuracy: 0.5648 - val_loss: 0.6296 - val_accuracy: 0.5898
Epoch 3/10
3051/3051 [==============================] - 10s 3ms/step - loss: 0.0864 - accuracy: 0.5708 - val_loss: 0.6952 - val_accuracy: 0.5178
Epoch 4/10
3051/3051 [==============================] - 9s 3ms/step - loss: 0.0864 - accuracy: 0.5660 - val_loss: 0.6338 - val_accuracy: 0.5802
Epoch 5/10
3051/3051 [==============================] - 9s 3ms/step - loss: 0.0863 - accuracy: 0.5664 - val_loss: 0.6408 - val_accuracy: 0.5734
Epoch 6/10
3051/3051 [==============================] - 12s 4ms/step - loss: 0.0861 - accuracy: 0.5623 - val_loss: 0.6184 - val_accuracy: 0.6112
Epoch 7/10
3051/3051 [==============================] - 13s 4ms/step - loss: 0.0864 - accuracy: 0.5672 - val_loss: 0.6594 - val_accur